<div class="align-center">
<a href="https://oumi.ai/"><img src="https://oumi.ai/docs/en/latest/_static/logo/header_logo.png" height="200"></a>

[![Documentation](https://img.shields.io/badge/Documentation-latest-blue.svg)](https://oumi.ai/docs/en/latest/index.html)
[![Discord](https://img.shields.io/discord/1286348126797430814?label=Discord)](https://discord.gg/oumi)
[![GitHub Repo stars](https://img.shields.io/github/stars/oumi-ai/oumi)](https://github.com/oumi-ai/oumi)
</div>

👋 Welcome to Open Universal Machine Intelligence (Oumi)!

🚀 Oumi is a fully open-source platform that streamlines the entire lifecycle of foundation models - from [data preparation](https://oumi.ai/docs/en/latest/resources/datasets/datasets.html) and [training](https://oumi.ai/docs/en/latest/user_guides/train/train.html) to [evaluation](https://oumi.ai/docs/en/latest/user_guides/evaluate/evaluate.html) and [deployment](https://oumi.ai/docs/en/latest/user_guides/launch/launch.html). Whether you're developing on a laptop, launching large scale experiments on a cluster, or deploying models in production, Oumi provides the tools and workflows you need.

🤝 Make sure to join our [Discord community](https://discord.gg/oumi) to get help, share your experiences, and contribute to the project! If you are interested in joining one of the community's open-science efforts, check out our [open collaboration](https://oumi.ai/community) page.

⭐ If you like Oumi and you would like to support it, please give it a star on [GitHub](https://github.com/oumi-ai/oumi).

# Distillation Overview

In this tutorial, we'll show how we trained MiniMath-R1-1.5B!

We'll use the Oumi framework to streamline the process and achieve high-quality results.

We'll cover the following topics:
1. Prerequisites
2. Model and Data Preparation
3. Fine-Tuning
4. Evaluation
5. Upload to HuggingFace

# Prerequisites

## Hardware
The defaults in this tutorial are scaled down for demonstration purposes.

The true values are left to code comments within each section.

We recommend 8xA100-80GB GPUs to complete in a timely manner with adequate performance.

## Oumi Installation

First, let's install Oumi and vLLM. You can find more detailed instructions [here](https://oumi.ai/docs/en/latest/get_started/installation.html). Here, we include Oumi's GPU dependencies.


In [ ]:
%pip install oumi[gpu]

## Creating our working directory
For our experiments, we'll use the following folder to save the model, training artifacts, and our working configs.

In [ ]:
from pathlib import Path

tutorial_dir = "distillation_tutorial"

Path(tutorial_dir).mkdir(parents=True, exist_ok=True)

## Setup the environment

We'll need to set the following environment variables:
- [Optional] HF_TOKEN: Your [HuggingFace](https://huggingface.co/docs/hub/en/security-tokens) token, in case you want to access a private model.
- [Optional] WANDB_API_KEY: Your [wandb](https://wandb.ai) token, in case you want to log your experiments to wandb.

In [ ]:
import os

os.environ["HF_TOKEN"] = "INSERT TOKEN HERE"
os.environ["WANDB_API_KEY"] = "INSERT API KEY HERE"

# Model and Data Preparation

## Model Download

For our purposes it will be much faster if we download our models first.

We'll use the `hf_transfer` package to download.

In [ ]:
!pip install hf_transfer

In [ ]:
!HF_HUB_ENABLE_HF_TRANSFER=1 hf download \
    deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B --exclude original/*

# Baseline Evals

Before we can improve our small model, we should measure a baseline.

The below code will run the MMLU PRO Math task from LM Harness. 

Note that this will take some time, so we've recorded our results below for your convenience:

| Model | MMLU Pro Math Accuracy |
|-------|------------------------|
| R1 Distill 1.5B | 38.49% +- 1.32% |

### Run Evals

In [ ]:
%%writefile $tutorial_dir/eval_small.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
  torch_dtype_str: "bfloat16"
  # shard_for_eval: True # Uncomment this line for multi-gpu setups.


tasks:
  - evaluation_backend: lm_harness
    task_name: mmlu_pro_math

output_dir: "distillation_tutorial/output/evaluation"
generation:
  batch_size: 1 # LM Harness recommends BS=1 for reproducibility.
  # batch_size: 128  # Replace with 256 for 8xA100-80GB

In [ ]:
!oumi evaluate -c "$tutorial_dir/eval_small.yaml"

## Prepare Training Data

Oumi has released an Apache 2.0 license math dataset at `oumi-ai/MetaMathQA-R1`, let's go ahead and download it.

In [ ]:
!HF_HUB_ENABLE_HF_TRANSFER=1 hf download oumi-ai/MetaMathQA-R1 \
    --exclude original/* --repo-type dataset

# Fine-Tuning

Now that the data is downloaded, we can begin fine-tuning the model.

In [ ]:
%%writefile $tutorial_dir/train.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
  trust_remote_code: true
  torch_dtype_str: "bfloat16"
  model_max_length: 4096
  device_map: "auto"

data:
  train:
    datasets:
      - dataset_name: "PromptResponseDataset"
        split: "train"
        sample_count: 25000  # 25k samples is enough to get the desired effect
        dataset_kwargs: {
          "hf_dataset_path": "oumi-ai/MetaMathQA-R1",
          "prompt_column": "prompt",
          "response_column": "response",
        }
        shuffle: True
        seed: 42
    seed: 42

training:
  output_dir: "distillation_tutorial/output/finetune"

  # For a single GPU, the following gives us a batch size of 8
  # If training with multiple GPUs, feel free to reduce gradient_accumulation_steps
  per_device_train_batch_size: 1
  gradient_accumulation_steps: 8
  
  # ***NOTE***
  # We set it to 10 steps to first verify that it works
  # Comment out the line below to have it train for 1 full epoch (all the data) instead.
  # Note: 1 full epoch will take about 13 minutes on 8xH100-80GB.
  max_steps: 10
  num_train_epochs: 1
  learning_rate: 1e-4
  warmup_ratio: 0.1
  logging_steps: 10
  save_steps: 0
  max_grad_norm: 10
  weight_decay: 0.01
  compile: False

  
  trainer_type: "TRL_SFT"
  optimizer: "adamw_torch_fused"
  enable_gradient_checkpointing: True
  gradient_checkpointing_kwargs:
    use_reentrant: False
  ddp_find_unused_parameters: False
  dataloader_num_workers: "auto"
  dataloader_prefetch_factor: 32
  empty_device_cache_steps: 1

# Uncomment this for distributed training
# fsdp:
#   enable_fsdp: True
#   backward_prefetch: "BACKWARD_POST"
#   forward_prefetch: True
#   cpu_offload: True
#   auto_wrap_policy: "TRANSFORMER_BASED_WRAP"
#   transformer_layer_cls: "Qwen2DecoderLayer"

### Single GPU

In [ ]:
!oumi train -c "$tutorial_dir/train.yaml"

### Multi-GPU

In [ ]:
!oumi distributed torchrun -m oumi train -c "$tutorial_dir/train.yaml"

# Evaluation

Now that we have a new distilled model, let's evaluate it on the same benchmark.

In [ ]:
%%writefile $tutorial_dir/eval_small_fft.yaml

model:
  model_name: "./distillation_tutorial/output/"
  torch_dtype_str: "bfloat16"
  # shard_for_eval: True # Uncomment this line for multi-gpu setups.


tasks:
  - evaluation_backend: lm_harness
    task_name: mmlu_pro_math

output_dir: "distillation_tutorial/output/evaluation"
generation:
  batch_size: 1 # LM Harness recommends BS=1 for reproducibility.
  # batch_size: 256  # Replace with 256 for 8xA100-80GB

In [ ]:
!oumi evaluate -c "$tutorial_dir/eval_small_fft.yaml"

## Results

After we finetuned the model following the steps above, we achieved the following results:

| Model           | Accuracy        |
|-----------------|-----------------|
| R1 Distill 1.5B | 38.49% +- 1.32% |
| MiniMath R1 1.5B | 44.4% +- 1.34% |

# Upload to HuggingFace

After fine-tuning, let's upload our model to HuggingFace to make it easily portable to other places.

In [ ]:
HUGGINGFACE_REPO_PATH = "your-user-name/your-model-name"
LOCAL_MODEL_PATH = f"./{tutorial_dir}/output"

## Upload Model

Transformers makes it fairly easy to upload the model itself.

In [ ]:
import transformers

model = transformers.AutoModel.from_pretrained(LOCAL_MODEL_PATH, torch_dtype="bfloat16")
model.push_to_hub(HUGGINGFACE_REPO_PATH)

## Upload Configs

HuggingFace by default doesn't upload a number of important configs for inference so we
have to upload these manually.

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
model_files = [f for f in Path(LOCAL_MODEL_PATH).glob("*.json")]

for file in model_files:
    file_name = file.name
    api.upload_file(
        path_or_fileobj=file,
        path_in_repo=file_name,
        repo_id=HUGGINGFACE_REPO_PATH,
        repo_type="model",
    )